In [92]:
import pandas as pd

In [ ]:
'''
Objective: Given sorted network flow data, and demand, 
we would like to back trace processing steps 
(might be multiple splitted among countries) to source
'''
def traceback(networkFlow, demand, processing_steps):

    # get info on demand
    result = [{
        'Cnt' : demand['to_processing_cnt'],
        'Week' : demand['Week'],
        'Amount' : demand['Amount'],
        'Process' : f'{processing_steps[-1]}'
    }]
    
    send_from_cnt = demand['send_from_cnt']
    for process in reversed(processing_steps[:-1]):
        
        while True:

            rows = networkFlow[(networkFlow['to_processing_cnt'] == send_from_cnt) &
                        (networkFlow['for_process'] == process)]

            if rows.empty:
                break

            row = rows.iloc[0]
            result.append(row)

            country = row['send_from_cnt']
            process = row['for_process']  
            amount = row['Amount']

            if process == 'Sourcing':
                break

    result.reverse()

    return result




In [107]:
# Main
processing_steps = ['Sourcing', 'Conditioning', 'Treatment', 'Forwarding', 'Delivery']

# read input data
networkFlow = pd.read_excel('NetworkFlowProblem-Data.xlsx', sheet_name='Input6') 

# sort transaction based on week: assuming deliveries happen later at time
networkFlow_sorted = networkFlow.sort_values('Week', ascending=False)

# find demands
demands = networkFlow_sorted[networkFlow_sorted['for_process']==processing_steps[-1]]

demand = demands.loc[39,:]
# trace = traceback(networkFlow_sorted, demand, processing_steps)


In [111]:
# get info on demand
result = [{
    'Process' : f'{processing_steps[-1]}',
    'Cnt' : demand['to_processing_cnt'],
    'Week' : demand['Week'],
    'Amount' : demand['Amount'],

}]

send_from_cnt = demand['send_from_cnt']

for process in reversed(processing_steps[:-1]):
    rows = networkFlow[(networkFlow_sorted['to_processing_cnt'] == send_from_cnt) &
                    (networkFlow_sorted['for_process'] == process)]



[{'Process': 'Delivery',
  'Cnt': 'ROMANIA',
  'Week': 22,
  'Amount': 3488.23999999999}]

In [139]:
import pandas as pd
pd.set_option('display.max_columns', None)
def create_combined_dataframe(data_list, demand_identifier):
    
    process_order = ['Sourcing', 'Conditioning', 'Treatment', 'Forwarding', 'Delivery']
    process_dfs = {process: None for process in process_order}

    
    for d in reversed(data_list):
        process = d['Process']
        if process in process_order:
            
            if process_dfs[process] is None:
                process_dfs[process] = pd.DataFrame(columns=data_list[0].keys())
           
            process_dfs[process] = pd.concat([process_dfs[process], pd.DataFrame([d])], ignore_index=True)

    combined_df = pd.concat(process_dfs.values(), axis=1)
    combined_df['Demand'] = [f"{demand_identifier}-{counter+1}" for counter, _ in combined_df.iterrows()]

    return combined_df

# Example usage:
data_list = [
    {'Process': 'Delivery', 'Cnt': 'ROMANIA', 'Week': 22, 'Amount': 3488.23999999999},
    {'Process': 'Forwarding', 'Cnt': 'ITALY', 'Week': 22, 'Amount': 3488.23999999999},
    {'Process': 'Treatment', 'Cnt': 'ITALY', 'Week': 17, 'Amount': 1115.52999999998},
    {'Process': 'Treatment', 'Cnt': 'ITALY', 'Week': 15, 'Amount': 2372.71000000001},
    {'Process': 'Conditioning', 'Cnt': 'FRANCE', 'Week': 10, 'Amount': 3488.23999999999},
    {'Process': 'Sourcing', 'Cnt': 'FRANCE', 'Week': 1, 'Amount': 1500.0},
    {'Process': 'Sourcing', 'Cnt': 'FRANCE', 'Week': 1, 'Amount': 1988.2399999999898}
]

combined_df = create_combined_dataframe(data_list, 5)
combined_df.to_excel('alaki.xlsx')
combined_df


,Process,Cnt,Week,Amount,Process,Cnt,Week,Amount,Process,Cnt,Week,Amount,Process,Cnt,Week,Amount,Process,Cnt,Week,Amount,Demand
0,Sourcing,FRANCE,1,1988.24,Conditioning,FRANCE,10,3488.24,Treatment,ITALY,15,2372.71,Forwarding,ITALY,22,3488.24,Delivery,ROMANIA,22,3488.24,5-1
1,Sourcing,FRANCE,1,1500.00,NaN,NaN,NaN,NaN,Treatment,ITALY,17,1115.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5-2


In [142]:
import pandas as pd
pd.set_option('display.max_columns', None)

delta = 0.001

processing_steps = ['Sourcing', 'Conditioning',
                    'Treatment', 'Forwarding', 'Delivery']

def create_combined_dataframe(data_list, demand_identifier):
    
    process_order = ['Sourcing', 'Conditioning', 'Treatment', 'Forwarding', 'Delivery']
    process_dfs = {process: None for process in process_order}

    
    for d in reversed(data_list):
        process = d['Process']
        if process in process_order:
            
            if process_dfs[process] is None:
                process_dfs[process] = pd.DataFrame(columns=data_list[0].keys())
           
            process_dfs[process] = pd.concat([process_dfs[process], pd.DataFrame([d])], ignore_index=True)


    combined_df = pd.concat(process_dfs.values(), axis=1)
    combined_df['Demand'] = [f"{demand_identifier}-{counter+1}" for counter, _ in combined_df.iterrows()]

    return combined_df


# read input data
networkFlow = pd.read_excel(
    'NetworkFlowProblem-Data.xlsx', sheet_name='Input5')

# sort transaction based on week: assuming deliveries happen later at time
networkFlow_sorted = networkFlow.sort_values('Week', ascending=False)

# find demands
demands = networkFlow_sorted[networkFlow_sorted['for_process']
                             == processing_steps[-1]]

all_combined_dfs = pd.DataFrame()

counter = 0
for demand_identifier, demand in demands.iterrows():
    
    #demand = demands.loc[40, :]
    
    # get info on demand
    result = [{
        'Process': f'{processing_steps[-1]}',
        'Cnt': demand['to_processing_cnt'],
        'Week': demand['Week'],
        'Amount': demand['Amount'],
    
    }]
    
    send_from_cnt = [demand['send_from_cnt']]
    
    # for each process
    for process in reversed(processing_steps[:-1]):
        
        print(f"Demand: {demand['Amount']} @ {demand['to_processing_cnt']} at process: {process}")
        
        # check the ones hapenned at earlier weeks
        back_traces = networkFlow.loc[(networkFlow_sorted['to_processing_cnt'].isin(send_from_cnt)) &
                           (networkFlow_sorted['for_process'] == process) & 
                           (networkFlow_sorted['Week'] <= demand['Week'])]
        
        back_traces = back_traces.sort_values('Week', ascending=False) # I would like to sort based on week
        
        
        #check for the amount
        if (back_traces['Amount'].sum() + delta < demand['Amount']) | (back_traces.empty):
            print(f"Error 100: demand was {demand['Amount']}, greater than traced back flows, or no traces could be find for the demand.")
            break
        else:
            filled_demand = 0
            send_from_cnt = []
            for index, row in back_traces.iterrows():
                if filled_demand < demand['Amount']:
                    if filled_demand + row['Amount'] > demand['Amount']:
                        allocated_demand = demand['Amount'] - filled_demand;
                    else:
                        allocated_demand = row['Amount']
    
                    filled_demand += allocated_demand
                    send_from_cnt.append(row['send_from_cnt'])
                    result.append({
                        'Process': f'{process}',
                        'Cnt': row['to_processing_cnt'],
                        'Week': row['Week'],
                        'Amount': allocated_demand,
                        })
        counter += 1
        print(counter)
        combined_df = create_combined_dataframe(result, counter)
        print(combined_df.shape)
        #all_combined_dfs = pd.concat([all_combined_dfs,combined_df], ignore_index=True)  


#final_combined_df = pd.concat(all_combined_dfs, ignore_index=True, axis=0)



Demand: 673.099999999982 @ TURKEY at process: Forwarding
1
(1, 9)
Demand: 673.099999999982 @ TURKEY at process: Treatment
2
(2, 13)
Demand: 673.099999999982 @ TURKEY at process: Conditioning
3
(2, 17)
Demand: 673.099999999982 @ TURKEY at process: Sourcing
4
(2, 21)
Demand: 1854.53000000001 @ TURKEY at process: Forwarding
5
(3, 9)
Demand: 1854.53000000001 @ TURKEY at process: Treatment
6
(5, 13)
Demand: 1854.53000000001 @ TURKEY at process: Conditioning
7
(5, 17)
Demand: 1854.53000000001 @ TURKEY at process: Sourcing
8
(5, 21)
Demand: 6723.07 @ TURKEY at process: Forwarding
9
(1, 9)
Demand: 6723.07 @ TURKEY at process: Treatment
10
(1, 13)
Demand: 6723.07 @ TURKEY at process: Conditioning
11
(2, 17)
Demand: 6723.07 @ TURKEY at process: Sourcing
12
(2, 21)
Demand: 124.3 @ TURKEY at process: Forwarding
13
(1, 9)
Demand: 124.3 @ TURKEY at process: Treatment
14
(1, 13)
Demand: 124.3 @ TURKEY at process: Conditioning
15
(1, 17)
Demand: 124.3 @ TURKEY at process: Sourcing
16
(1, 21)


In [143]:
# Example data for used_resources and row
used_resources = pd.Series([1, 2, 3])
row = pd.Series([4, 5, 6])

# Concatenate vertically
concatenated_series = pd.concat([used_resources, row], ignore_index=True, axis=0)

concatenated_series

0    1
1    2
2    3
3    4
4    5
5    6
dtype: int64